In [21]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



<div style="color:lightgreen">
Eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

```
SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen
FROM dataFrame
WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?
HAVING COUNT(x) > 1
GROUP BY user_id;
```


Vorschlag Sebastian:
<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>
</div>

<div style="color:lightgreen">

Mein Vorschlag:

Ist es nicht besser direkt nach Nutzer und Zeit zu gruppieren? Dann hat man für jeden Nutzer direkt angezeigt, wie oft zu einem Zeitpunkt Pixel platziert wurden?
Dann wäre auch die Bereinigung der Daten effektiver als oben.

Hier der Versuch:
</div>

In [2]:
from src.features.feature_functions import group_dataframe_by_time_and_user
import pyspark.sql.functions as F

timeUserGrouped = group_dataframe_by_time_and_user(dataFrame)
timeUserGrouped.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [3]:
timeUserGrouped = timeUserGrouped.withColumnRenamed('count','time_count')

timeUserGrouped = timeUserGrouped.filter(timeUserGrouped.time_count > 1)
timeUserGrouped.show()

+--------------------+-----+----------+--------------------+--------------------+-------------------------+
|             user_id|    t|time_count|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+--------------------+-----+----------+--------------------+--------------------+-------------------------+
|X8gpDfO6+Z87SKKTy...|29312|         8|[1685, 904, 932, ...|[228, 1865, 1850,...|     [#FFD635, #000000...|
|AxBTmhBMbr1sbDw/2...|28682|         6|[1403, 777, 1741,...|[2, 8, 10, 2, 8, 10]|     [#FFA800, #FFA800...|
|DIECtVqoJqBlL6A0C...|27538|         6|[932, 920, 933, 9...|[1757, 1832, 1838...|     [#FFFFFF, #BE0039...|
|AxBTmhBMbr1sbDw/2...|28064|         6|[655, 612, 1575, ...|[1896, 1917, 7, 1...|     [#B44AC0, #E4ABFF...|
|tSD0qKwzfmmglNjOQ...|28913|         4|[941, 913, 941, 913]|[1869, 1865, 1869...|     [#FFFFFF, #51E9F4...|
|AxBTmhBMbr1sbDw/2...|27139|         4|[1287, 1926, 1287...|      [8, 19, 8, 19]|     [#3690EA, #BE0039...|
|SXF5Vf5UlWU1UW/0F...|27967|

In [4]:
grouped = timeUserGrouped.dropDuplicates(['user_id'])
grouped.show()

+--------------------+-----+----------+---------------+---------------+-------------------------+
|             user_id|    t|time_count|collect_list(x)|collect_list(y)|collect_list(pixel_color)|
+--------------------+-----+----------+---------------+---------------+-------------------------+
|++++KubvRsRI2sZI3...|28165|         2|   [1434, 1434]|     [360, 360]|       [#9C6926, #9C6926]|
|++0nCmngtZ68vbg6p...|27072|         2|     [757, 757]|     [592, 592]|       [#000000, #000000]|
|++3Ki0p8wbp0g7xQv...|29762|         2|   [1975, 1975]|       [92, 92]|       [#FFFFFF, #FFFFFF]|
|++3gcvcX6sgZwlgO7...|28565|         2|     [983, 983]|   [1529, 1529]|       [#000000, #000000]|
|++3l0k1HSXKiAsHkH...|27719|         2|     [668, 668]|   [1211, 1211]|       [#D4D7D9, #D4D7D9]|
|++4/asfR6j/RbsvIF...|29079|         2|     [467, 467]|     [937, 937]|       [#FFFFFF, #FFFFFF]|
|++4PiR2T1Y0vPxBnK...|29843|         2|     [205, 205]|   [1852, 1852]|       [#000000, #000000]|
|++5bQtHNJxwF1lYE2..

In [5]:
print(grouped.count())

22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/07 14:53:36 WARN RowBasedKeyValueBatch: Calling spill() on

733580


In [4]:
row = dataFrame.count('user_id')
print(f'Anzahl der herkömmlichen Zeilen: {row}')

row_sel = timeUserGrouped.count('user_id')
print(f'Anzahl der angeblichen Moderatoren: {row_sel}')

TypeError: count() takes 1 positional argument but 2 were given

<div style="color:lightgreen">
Vllt doch besser nur über Zeit zu gruppieren?
</div>

In [22]:
from src.features.feature_functions import group_dataframe_by_time

timeGrouped = group_dataframe_by_time(dataFrame)
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [23]:
timeGrouped = timeGrouped.withColumnRenamed('count','time_count')
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- time_count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [24]:
timeGrouped = timeGrouped.filter(timeGrouped.time_count > 1)
timeGrouped.show()

+----+----------+---------------------+--------------------+--------------------+-------------------------+
|   t|time_count|collect_list(user_id)|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+----+----------+---------------------+--------------------+--------------------+-------------------------+
|1576|      2670| [HCpm+67Bk9Hnia1k...|[151, 0, 29, 1780...|[1884, 1672, 1815...|     [#000000, #000000...|
|1573|      2670| [opKWRQuucm7TrBLu...|[44, 120, 0, 1777...|[1713, 1737, 1120...|     [#000000, #FFA800...|
|1575|      2639| [hnp8WrkaEwMXOvXJ...|[154, 143, 89, 11...|[1867, 1696, 1675...|     [#000000, #000000...|
|1574|      2593| [2IbpXz/9Gtjfgfo5...|[182, 189, 42, 13...|[1761, 1722, 1708...|     [#000000, #BE0039...|
|1578|      2591| [3vSAHWRA4lxx0P73...|[104, 224, 62, 14...|[1860, 1790, 1957...|     [#000000, #BE0039...|
|1577|      2560| [MF1wnz7k+eu72s6l...|[180, 208, 48, 18...|[227, 1749, 1740,...|     [#FFA800, #000000...|
|1579|      2500| [6TUrLNzZB

<div style="color:lightgreen">
Jz muss für jeden Zeitpunkt die Userliste betrachtet werden.

Dabei muss geschaut werden, welcher User mehrmals vorkommt.

D.h. Vorkommen zählen.
</div>

<div style="color:#cb4861">
Neuer Ansatz:

Self-Join über Nutzer & quasi den selben Zeitpunkt, dann sagen, dass t1 < t2 sein soll.
Anschließend schauen ob Nutzer mehrfach vorkommt.
</div>

In [25]:
import pyspark.sql.functions as F

selfJoinTime = dataFrame.alias('t1').join(dataFrame.alias('t2'), F.col('t1.user_id') == F.col('t2.user_id'),'inner')
selfJoinTime = selfJoinTime.where('t1.t == t2.t').where('t1.x < t2.x OR t1.y < t2.y')
selfJoinTime.distinct().show()

+--------------------+----+----+-----+-----------+--------------------+----+----+-----+-----------+
|             user_id|   x|   y|    t|pixel_color|             user_id|   x|   y|    t|pixel_color|
+--------------------+----+----+-----+-----------+--------------------+----+----+-----+-----------+
|Qafq4ohzHG2/BRgOZ...|1757| 919|26613|    #000000|Qafq4ohzHG2/BRgOZ...| 948|1877|26613|    #FFA800|
|Qafq4ohzHG2/BRgOZ...| 948|1877|26613|    #FFA800|Qafq4ohzHG2/BRgOZ...|1757| 919|26613|    #000000|
|i6Hv8krtoGNymbs3c...| 916|1848|27172|    #51E9F4|i6Hv8krtoGNymbs3c...| 938|1840|27172|    #FFFFFF|
|i6Hv8krtoGNymbs3c...| 938|1840|27172|    #FFFFFF|i6Hv8krtoGNymbs3c...| 916|1848|27172|    #51E9F4|
|SXF5Vf5UlWU1UW/0F...| 926|1832|27967|    #BE0039|SXF5Vf5UlWU1UW/0F...| 930|1847|27967|    #51E9F4|
|vw/fG+wFQWr02Vraf...|1647| 126|28269|    #7EED56|vw/fG+wFQWr02Vraf...|1650| 127|28269|    #000000|
|SXF5Vf5UlWU1UW/0F...| 981|1844|28807|    #000000|SXF5Vf5UlWU1UW/0F...| 991|1840|28807|    #9C6926|


In [26]:
selfJoinTime.select('t1.user_id').distinct().show()

+--------------------+
|             user_id|
+--------------------+
|SXF5Vf5UlWU1UW/0F...|
|Qafq4ohzHG2/BRgOZ...|
|i6Hv8krtoGNymbs3c...|
|vw/fG+wFQWr02Vraf...|
|BmjXARzU+xyFQLSPJ...|
|E7dwXBrzqR3gjN1LF...|
|os0SvQIUe7Ezdo+UM...|
|FqWlYosC7Oca6Vvqp...|
|K+xJuAvIjsDUc9iPJ...|
|kYhL/Z4Y2RcSpKmTD...|
|LPGe0TJTM3jja4dtJ...|
|Yosa4ArHwnXDGUyZJ...|
|C5UJR4VvOj3EJTEEb...|
|MTsePTaE/v1L9+xpz...|
|AxBTmhBMbr1sbDw/2...|
|6dbHlCcUjHtb8ZwtH...|
|n6JR4vSag3sih2kpE...|
|HpaGMZJYSrz/K5y8f...|
|WjSSJUeYPQxt8+3yR...|
|UHCTBiKzVoFjs/Peh...|
+--------------------+
only showing top 20 rows



<div style="color:#cb4861">
Eventuell sind das nun wirklich alle Moderatoren...
</div>

In [27]:
mods = selfJoinTime.select('t1.user_id').distinct().collect()
mod = []

for row in mods:
    mod.append(list(row))

In [28]:
selfJoinTime.select('t1.user_id').distinct().count()

98

<div style="color:#cb4861">
Angeblich haben wir im Datensatz 51 folgende Anzahl an Mods: 98
</div>

In [29]:
for i in mod:
    print(i)

['SXF5Vf5UlWU1UW/0FtsA+EBcgjJpb8kseydlLzfWRbD024834DAUQMzi8qgpIvI/hoIf7yw9rd+RLCCHD4eZ8g==']
['Qafq4ohzHG2/BRgOZP3UKYSAnmaLB4txrV82QG57+GkOx4Xf0lK0Qr06Ym69Xq3XB79O+KMIV8BSySnHwgJA9Q==']
['i6Hv8krtoGNymbs3c8iWVSq7AzoV0vUvOMbNU0GvQyC4klKLPBVigV/JecsN39+PQfR4QFzLBNf1vJZw5ZlW1A==']
['vw/fG+wFQWr02VrafWaTOE2lF/6+Gr2nq2Sb6t9TUUc4UlAZEMnh06KYG5Z7uuQGwjGJGwKCnWiEccdFNa/GRg==']
['BmjXARzU+xyFQLSPJcgItYh/JTWExwfb6rtGN4lPAMCNXzIgfw1oLOaInLfX0z2OLS0Cya5YmRaNL63Cx/cMew==']
['E7dwXBrzqR3gjN1LFc4crkPmPukLP7vs4g6O8YQSz2dhIwLD7gUOYRVqOwsfl+/o/rvYl7zWDwS8m+7Cn7zNfQ==']
['os0SvQIUe7Ezdo+UMmzj3F9EnRQ3Ma6F/jfcUKSExtbdxJdux+sPzZxJ7+2Q5dC1hX8uexhj5KUyggFep5aIwA==']
['FqWlYosC7Oca6Vvqp16Gwjs7g4vVTr2BPRUggGnTaRrr46IrtkzTK9nGYqIdw1R6rglsHydp7VZQmhEN4w7Zlg==']
['K+xJuAvIjsDUc9iPJiLJCusV2YfAVPWhgIvLKBsXjZ4o8T0Xq8dUxYO1oUfZqjhUxOn1++pvI59mE4gx4STihw==']
['kYhL/Z4Y2RcSpKmTDoNrfCfhbbyE70y7ORonAbJWYylWlf6IBaCZ8tok8rogu2syFcBIRrQGUDjHfGTeIh22aw==']
['LPGe0TJTM3jja4dtJaP5tRL4Dp1fvIab9VENNoUh7j/7NbK4U1fiA/Cji6N4DRgYyrd3

In [10]:
from pyspark.sql.window import Window

time = dataFrame.select('*', F.collect_list(dataFrame.user_id).over(Window.partitionBy('t').orderBy('t')).alias('grouped')).show()

+--------------------+----+----+---+-----------+--------------------+
|             user_id|   x|   y|  t|pixel_color|             grouped|
+--------------------+----+----+---+-----------+--------------------+
|IAAh6iWxJGaf1oRm9...|1809|1267|  0|    #FFFFFF|[IAAh6iWxJGaf1oRm...|
|9N16/SSHrZHFYJQ7J...| 349|1367|  0|    #FFD635|[IAAh6iWxJGaf1oRm...|
|vA+Y4DI6U+VaV6mV4...| 436| 688|  0|    #6D001A|[IAAh6iWxJGaf1oRm...|
|KcbN7QDEAXzyWiU+m...|  93|1565|  0|    #FFFFFF|[IAAh6iWxJGaf1oRm...|
|YrIQFGoonXdkvhtZh...|  91|1431|  0|    #FFFFFF|[IAAh6iWxJGaf1oRm...|
|I/NZyXUfBFL0TMi4U...| 142|1392|  0|    #FFFFFF|[IAAh6iWxJGaf1oRm...|
|eMPTc99SeWVocVEEC...| 906| 596|  0|    #D4D7D9|[IAAh6iWxJGaf1oRm...|
|o9KAXWkMVk0OHxDhm...| 963|1014|  0|    #2450A4|[IAAh6iWxJGaf1oRm...|
|/YlplpLkn1LWaKsPk...|1491| 425|  0|    #000000|[IAAh6iWxJGaf1oRm...|
|7xeRfKvCtScl/SJt1...|1883|1185|  0|    #898D90|[IAAh6iWxJGaf1oRm...|
|bN8SueTAmrhyH7IuZ...| 339|1179|  0|    #FFFFFF|[IAAh6iWxJGaf1oRm...|
|D1/xXtaq13nImIUxH..

<div style="color:#b0e0e6">
Alle Spalten bis auf 't' und 'collect_list(user_id)' können entfernt werden, da unwichtig.
</div>

In [9]:
timeDroped = timeGrouped.drop('time_count').drop('collect_list(x)').drop('collect_list(y)').drop('collect_list(pixel_color)')
timeDroped.printSchema()

root
 |-- t: long (nullable = true)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [10]:
timeDroped.show()

+----+---------------------+
|   t|collect_list(user_id)|
+----+---------------------+
|1576| [HCpm+67Bk9Hnia1k...|
|1573| [opKWRQuucm7TrBLu...|
|1575| [hnp8WrkaEwMXOvXJ...|
|1574| [2IbpXz/9Gtjfgfo5...|
|1578| [3vSAHWRA4lxx0P73...|
|1577| [MF1wnz7k+eu72s6l...|
|1579| [6TUrLNzZBX5PjodW...|
|1571| [4IpCxhkBwhPQpR4z...|
|1580| [crx8iMrGIkGP57xg...|
|1572| [oaeb4agaHZiqaCh+...|
|1570| [RkARTAsTKwgf5n0/...|
|1581| [iWlP3jeQPASmW+TJ...|
|1560| [y2uAe1fKExqOPuuh...|
|1585| [2VXxrykKv3MmhgAG...|
|1583| [MUo7SggrxHsKAmif...|
|1584| [10Gm0bGFon6ciTm6...|
|1161| [2notc2uVA+WK6TIx...|
|1569| [uS+hbQGNmBfaa0dM...|
|1164| [QCxllnKX36d7cWja...|
|1582| [OB8ScyBUpVGAy4aF...|
+----+---------------------+
only showing top 20 rows



In [11]:
collected = timeDroped.collect()
count = 0
moderator = []

for row in collected:
    time,users = list(row)
    for user in users:
        for comp in users:
            if (user == comp): 
                count +1
            if(user in moderator) : 
                break #in äußere Spalte zurückkehren
            if(count >= 2) : 
                moderator.append(user)
                break #in äußere Spalte zurückkehren
        count = 0
print(moderator)   

22/06/07 14:54:33 ERROR Utils: Uncaught exception in thread task-result-getter-1
java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2083)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1657)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2431)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2213)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1669)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2119)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1657)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:503)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:461)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:76)
	at org.

ConnectionRefusedError: [Errno 111] Connection refused

<div style="color:#b0e0e6">
Es ergibt sich eine leere Liste, ist das so korrekt?
Denn das würde bedeuten, es gäbe in der einen Stunde keine Moderatoren...
(Siehe auch Anmerkung unten)
</div>

In [12]:
dataFrame.orderBy(['user_id'], ascending = [True]).show()

ConnectionRefusedError: [Errno 111] Connection refused

<div style="color:#b0e0e6">
Warum gibt's bei manchen Usern Dopplungen der selben Zeit in Kombo mit den selben Koordinaten und bei manchen nicht?
Deutet das auf Moderatoren hin?
Evtl. müsste man den Datensatz selbst im Original nochmal anschauen.

Was ich mich auch frage ist, ob die Aufspaltung in x und y Koordinaten so korrekt ist,
da Moderatoren x1,x2,y1,y2 Angaben in der Spalte haben und nicht x,y....
Also ob die Anpassung des Datensatzes, wie Sebastian es vorbereitet hat, so passend ist mit den Moderatoren?
</div>